In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import numpy as np
import pandas as pd

from datasets import LoadData, ImageDataset

ds = LoadData('./data')

label_df = ds.load_dataset()
image_df = ds.load_image()

label_df

,TCGA ID,censored,Survival months,Grade,Age at diagnosis,Molecular subtype
0,TCGA-02-0001,0,358,4.0,44,IDHwt
1,TCGA-02-0003,0,144,4.0,50,IDHwt
2,TCGA-02-0006,0,558,4.0,56,IDHwt
3,TCGA-02-0007,0,705,4.0,40,IDHwt
4,TCGA-02-0009,0,322,4.0,61,IDHwt
...,...,...,...,...,...,...
764,TCGA-WY-A85A,1,1320,NaN,20,IDHmut-non-codel
765,TCGA-WY-A85B,1,1393,NaN,24,IDHmut-non-codel
766,TCGA-WY-A85C,1,1426,NaN,36,IDHmut-non-codel
767,TCGA-WY-A85D,1,1147,NaN,60,IDHmut-non-codel


In [2]:
import tqdm
import torch

def prediction(model, data_loader, device):
    model.eval()
    survtime_preds = []
    for sample_batch in data_loader:
        image_batch = sample_batch['image'].to(device, dtype=torch.float)
        
        with torch.no_grad():
            survtime_preds_batch = model(image_batch)
        survtime_preds.append(survtime_preds_batch)
    survtime_preds = torch.cat(survtime_preds)
    
    return survtime_preds

In [3]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index

from datasets import PretrainDataset
from models import PretrainNet
from train import Trainer
from utils.utils import CoxLoss


image_path = './data/all_st_patches_512'
BATCH_SIZE = 128
device = 'cuda:0'

rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=None)
avg_agg = []
max_agg = []
min_agg = []

for i, (training_idx, test_idx) in enumerate(rkf.split(label_df)):
    print('{}/{}'.format(i+1, rkf.get_n_splits()))
    
    # train val test set
    training_set = label_df.iloc[training_idx, :]
    test_set = label_df.iloc[test_idx, :]
    train_set, val_set = train_test_split(training_set, test_size=0.1)

    train_df = train_set.merge(ds.load_image(), on = 'TCGA ID')
    val_df = val_set.merge(ds.load_image(), on = 'TCGA ID')
    test_df = test_set.merge(ds.load_image(), on = 'TCGA ID')
    
    
    # Datasets
    train_dataset = PretrainDataset(dataframe = train_df, root_dir = image_path,
                                    transform = transforms.Compose([transforms.RandomHorizontalFlip(0.5),
                                                                    transforms.RandomVerticalFlip(0.5),
                                                                    transforms.ColorJitter(brightness=0.1, contrast = 0.1),
                                                                    transforms.Resize(224),
                                                                    transforms.ToTensor()]))

    val_dataset = PretrainDataset(dataframe = val_df, root_dir = image_path,
                                  transform = transforms.Compose([transforms.Resize(224),
                                                                  transforms.ToTensor()]))
    
    test_dataset = PretrainDataset(dataframe=test_df, root_dir=image_path,
                                   transform=transforms.Compose([transforms.Resize(224),
                                                                 transforms.ToTensor()]))
    
    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
    val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False)
    test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False)
    
    model = PretrainNet()
    model.to(device)
    
    trainer = Trainer(model, train_loader, val_loader, criterion = CoxLoss, optimizer = optim.Adam,
                      epochs = 50, lr = 0.0005, device = device)
    trainer.fit()
    best_model = trainer.get_best_model()
    
    hazards = torch.exp(prediction(best_model, test_loader, device))
    test_df['hazards'] = np.array(hazards.to('cpu'))
    
    # Aggregation
    tcga_id_list = list(test_df['TCGA ID'].unique())
    result = pd.DataFrame(columns=['TCGA ID', 'censored', 'Survival months', 
                                   'hazard_avg', 'hazard_max', 'hazard_min'])
    for tcga_id in tcga_id_list:
        df = test_df[test_df['TCGA ID'] == tcga_id].reset_index(drop=True)
        
        censored = df.loc[0, 'censored']
        survival_months = df.loc[0, 'Survival months']
        hazard_avg = df['hazards'].mean()
        hazard_max = df['hazards'].max()
        hazard_min = df['hazards'].min()
        result = result.append({'TCGA ID' : tcga_id,
                                'censored' : censored, 
                                'Survival months' : survival_months,
                                'hazard_avg' : hazard_avg,
                                'hazard_max' : hazard_max,
                                'hazard_min' : hazard_min}, ignore_index=True)
        
    avg_cindex = concordance_index(result['Survival months'], -result['hazard_avg'], 1-result['censored'])
    max_cindex = concordance_index(result['Survival months'], -result['hazard_max'], 1-result['censored'])
    min_cindex = concordance_index(result['Survival months'], -result['hazard_min'], 1-result['censored'])
    
    print('avg aggregation : {:.4f}, max aggregation : {:.4f}, min aggregation : {:.4f}'.format(avg_cindex, max_cindex, min_cindex))
    
    avg_agg.append(avg_cindex)
    max_agg.append(max_cindex)
    min_agg.append(min_cindex)
    print('-'*30)

1/15


  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

epoch: 1/50 
train_cindex: 0.6086, val_loss: 1.8621, val_cindex: 0.6347


  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

epoch: 2/50 
train_cindex: 0.6637, val_loss: 1.8887, val_cindex: 0.5768


...

epoch: 48/50 
train_cindex: 0.7727, val_loss: 2.0857, val_cindex: 0.6605


  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

epoch: 49/50 
train_cindex: 0.7785, val_loss: 2.7964, val_cindex: 0.6290


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [05:10<00:00,  3.98s/it]


epoch: 50/50 
train_cindex: 0.7779, val_loss: 2.1276, val_cindex: 0.5925
Training complete in 276m 37s
avg aggregation : 0.6698, max aggregation : 0.6432, min aggregation : 0.6762
------------------------------


In [4]:
df = pd.DataFrame({'avg_agg' : avg_agg,
                  'max_agg' : max_agg,
                  'min_agg' : min_agg})
df

,avg_agg,max_agg,min_agg
0,0.714329,0.693495,0.696493
1,0.725709,0.707041,0.700867
2,0.717983,0.723864,0.678430
3,0.627420,0.644843,0.582365
4,0.698866,0.678156,0.694029
5,0.643130,0.630750,0.632279
6,0.675535,0.687489,0.630072
7,0.731849,0.726153,0.704160
8,0.732451,0.708012,0.739434
9,0.725914,0.692553,0.714978


In [5]:
df.mean()

avg_agg    0.697785
max_agg    0.691782
min_agg    0.674011
dtype: float64

In [6]:
df.to_excel('result/simple aggregation_4.xlsx')